In [2]:
import os
import bs4
import json
import time
import lxml
import base64
import requests
from google.cloud import texttospeech_v1beta1
from IPython.display import clear_output as clear

In [3]:
top_dir = os.path.abspath('').replace('corrections', '')
conjugations_data_dir = os.path.join(top_dir, 'conjugations', 'data', 'language-specific')

In [4]:
headers = {"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
           "accept-encoding": "gzip, deflate, br",
           "accept-language": "en-GB,en;q=0.9,es-ES;q=0.8,es;q=0.7,en-US;q=0.6,eu;q=0.5",
           "cache-control": "max-age=0",
           "cookie": "t=238707487; _ga=GA1.2.1376835774.1641262578; _gid=GA1.2.1482423077.1641262578; _fbp=fb.1.1641262579526.851471446",
           "referer": "https://hidemy.name/en/proxy-list/?start=64",
           "sec-ch-ua-mobile": "?0",
           "sec-ch-ua-platform": "macOS",
           "sec-fetch-dest": "document",
           "sec-fetch-mode": "navigate",
           "sec-fetch-site": "same-origin",
           "sec-fetch-user": "?1",
           "upgrade-insecure-requests": "1",
           "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"}

In [5]:
language = 'spanish'

In [6]:
json_path = os.path.join(conjugations_data_dir, f'conjugations_{language}.json')

with open(json_path, 'r', encoding = 'utf8') as file:
    conjugations = json.loads(file.read())

infinitives = [(verb, conjugations[verb]['rank']) for verb in conjugations if conjugations[verb]['rank'] < 2001]

In [7]:
# data = []

# for n in range (0, len(infinitives)):

#     verb, rank = infinitives[n]

#     try:
#         count = {}

#         url = f'https://context.reverso.net/translation/{language}-english/{verb}'
#         page = requests.get(url, headers = headers)
#         soup = bs4.BeautifulSoup(page.text, 'lxml')

#         for i in range(0, len(soup.select('#translations-content .translation .pos-mark span'))):
            
#             trans_type = str(soup.select('#translations-content .translation .pos-mark span')[i])[13]

#             if trans_type in count.keys():
#                 count[trans_type] += 1
#             else:
#                 count[trans_type] = 1

#             if i == 0:
#                 if trans_type == 'v':
#                     primary = True
#                 else:
#                     primary = False

#         data.append([verb, rank, primary, count])
        
#         status = 'complete'
    
#     except Exception as ex:
#         print(ex)
#         data.append([verb, rank, 'translations not found'])
#         status = 'error'

#     clear(); print(f'{verb} ({rank}) {status} - {round(n/20, 3)}%')

#     time.sleep(0.1)

# non_primary = [x for x in data if not x[2]]

In [8]:
# non_primary = [x for x in data if not x[2]]
# errors = [x for x in data if 'translations not found' in x]
# lean = []

# for item in data:

#     if item[0] not in [x[0] for x in errors]:
    
#         verb,rank,primary,count = item

#         if primary:
#             if 'v' in count: v = count['v']
#             else: v = 0

#             nv = 0

#             for t in count:
#                 if t != 'v':
#                     nv += count[t]
        
#         if nv > v:
#             lean.append([verb, rank, (v, nv)])

# print(f'errors: {errors}')
# print(f'lean: {lean}')

# for item in non_primary:
#     verb,rank,_,count = item
#     print(verb, rank, count, conjugations[verb]['translations'][0])

In [9]:
rerank = {
    'spanish':      {'deber': 12, 'abrir': 60, 'pesar': 79, 'andar': 90, 'regular': 600, 'auxiliar': 700},
    'french':       {'conseiller': 400, 'rire': 250, 'officier': 750, 'tendre': 450},
    'german':       {'werden': 4, 'aufkommen': 1400, 'schlichten': 1500, 'vergehen': 1600, 'ausscheiden': 1700, 
                    'verkommen': 1800, 'anwachsen': 1900},
    'italian':      {'popolare': 750, 'parere': 250, 'regolare': 300, 'ammontare': 450, 'consolare': 760, 
                    'capitolare': 1500, 'ardire': 1800},
    'portuguese':   {'poder':8, 'jantar':110, 'dever':15, 'colar':510, 'amanhecer':1500, 'colher':900, 
                    'circular':1600, 'disciplinar':1400,}
}

In [10]:
dispose = {
    'spanish':      ['placer', 'solar', 'linear', 'protocolar', 'titular', 'tutelar', 'cautelar', 'nuclear', 'vehicular', 'militar',
                    'aver', 'trasmitir', 'substituir', 'recebir', 'remplazar', 'reinterpretar', 'reestablecer', 'efectivizar'],
    'french':       ['décrocher', 'repentir', 'souvenir', 'fier', 'réfugier', 'soucier', 'méfier', 'suicider', 
                    'méprendre', 'targuer', 'lamenter', 'rocher', 'plancher', 'clocher', 'latter', 'manager', 'bûcher', 'corner'],
    'german':       ['sondern', 'einigen', 'bezogen', 'geblieben', 'gingen', 'sieben', 'begonnen', 'berufen', 'überlegen', 
                    'modern', 'vermögen', 'verfahren', 'verborgen', 'angefangen', 'rücken', 'eingehalten', 'bescheiden', 
                    'angekommen', 'grauen', 'abgezogen', 'unterlegen', 'eingetroffen', 'gediehen', 'eingefangen', 'verschlagen', 
                    'zutrauen', 'zugegriffen', 'ausersehen', 'beschossen', 'geraten', 'verhalten', 'vorbehalten', 'verlegen', 
                    'eindringen', 'entgegenkommen', 'benehmen'],
    'italian':      ['scolare', 'militare', 'parlamentare', 'cautelare', 'regolamentare', 'calcare', 'mobiliare', 'domiciliare', 
                    'collare', 'fire', 'nettare', 'granulare'],
    'portuguese':   ['militar', 'nuclear', 'solar', 'anoitecer', 'pilar', 'glaciar', 'titular', 'placar', 'jugular', 
                    'suplementar', 'gear', 'domiciliar', 'anelar', 'velar']
}  

In [11]:
insert = []

for verb in rerank[language]:
    insert.append(conjugations.pop(verb))

In [12]:
for verb in dispose[language]:
    conjugations.pop(verb)

In [13]:
conjugations_list = []

for verb in conjugations:
    conjugations_list.append(conjugations[verb])

del conjugations

In [14]:
for verb in insert:
    for i in range(0, len(conjugations_list)):
        if conjugations_list[i]['rank'] == rerank[language][verb['infinitive']]:
            conjugations_list.insert(i, verb)
            break

In [15]:
new_conjugations = {}

for i in range(0, len(conjugations_list)):
    verb = conjugations_list[i]
    verb['rank'] = i + 1

    new_conjugations[verb['infinitive']] = verb

print(len(new_conjugations))
print(new_conjugations[conjugations_list[-1]['infinitive']]['rank'])

del conjugations_list

6782
6782


In [16]:
with open(os.path.join(conjugations_data_dir, f'conjugations_{language}.json'), 'w', encoding = 'utf8') as file:
    json.dump(new_conjugations, file, indent = 4, ensure_ascii = False)

In [17]:
# infinitives = [(verb, new_conjugations[verb]['rank']) for verb in new_conjugations if new_conjugations[verb]['rank'] < 2001]
# data = []

# for n in range (1900, len(infinitives)):

#     verb, rank = infinitives[n]

#     try:
#         count = {}

#         url = f'https://context.reverso.net/translation/{language}-english/{verb}'
#         page = requests.get(url, headers = headers)
#         soup = bs4.BeautifulSoup(page.text, 'lxml')

#         for i in range(0, len(soup.select('#translations-content .translation .pos-mark span'))):
            
#             trans_type = str(soup.select('#translations-content .translation .pos-mark span')[i])[13]

#             if trans_type in count.keys():
#                 count[trans_type] += 1
#             else:
#                 count[trans_type] = 1

#             if i == 0:
#                 if trans_type == 'v':
#                     primary = True
#                 else:
#                     primary = False

#         data.append([verb, rank, primary, count])
        
#         status = 'complete'
    
#     except Exception as ex:
#         print(ex)
#         data.append([verb, rank, 'translations not found'])
#         status = 'error'

#     clear(); print(f'{verb} ({rank}) {status} - {round(n/20, 3)}%')

#     time.sleep(0.1)

# non_primary = [x for x in data if not x[2]]

In [18]:
# non_primary = [x for x in data if not x[2]]
# errors = [x for x in data if 'translations not found' in x]
# lean = []

# for item in data:

#     if item[0] not in [x[0] for x in errors]:
    
#         verb,rank,primary,count = item

#         if primary:
#             if 'v' in count: v = count['v']
#             else: v = 0

#             nv = 0

#             for t in count:
#                 if t != 'v':
#                     nv += count[t]
        
#         if nv > v:
#             lean.append([verb, rank, (v, nv)])

# print(f'errors: {errors}')
# print(f'lean: {lean}')

# for item in non_primary:
#     verb,rank,_,count = item
#     print(verb, rank, count, new_conjugations[verb]['translations'][0])